In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [79]:
df = pd.read_excel('data_full_5sekres.xlsx')

In [80]:
df1 = df.copy()


# Firsly create a time series total load profile and generation

In [81]:
#Create a column of load profile
df1['BaseLoad'] = 300
df1.loc[df1['operationId'] == 100, 'BaseLoad'] = 500
df1.loc[df1['operationId'] == 3, 'BaseLoad'] = 550
df1.loc[df1['operationId'] == 5, 'BaseLoad'] = 510
df1.loc[df1['operationId'] == 6, 'BaseLoad'] = 250
df1.loc[df1['operationId'] == 1, 'BaseLoad'] = 250
df1.loc[df1['operationId'] == 7, 'BaseLoad'] = 380
df1['Load'] = df1['thrusterAndPropulsion'] + df1['BaseLoad']

In [82]:
#Create dataset with only relevant columns
df2 = df1[['datetime', 'Load', 'engMdoPower_0_','engMdoPower_1_','engMdoPower_2_', 'engMdoPower_3_', 'operationId', 'BaseLoad']]

#Set engine powers to zero
num = df2['engMdoPower_0_']._get_numeric_data()
num[num != 0] = 0

num = df2['engMdoPower_1_']._get_numeric_data()
num[num != 0] = 0

num = df2['engMdoPower_2_']._get_numeric_data()
num[num != 0] = 0

num = df2['engMdoPower_3_']._get_numeric_data()
num[num != 0] = 0



In [96]:
df4 = df2.copy()

load = df4['Load'].values
dg0 = df4['engMdoPower_0_'].values
dg1 = df4['engMdoPower_1_'].values
dg2 = df4['engMdoPower_2_'].values
dg3 = df4['engMdoPower_3_'].values
opid = df4['operationId'].values

for i in range(len(load)):
    if opid[i] == 100:
        if load[i] <= 2100:
            dg0[i] = dg1[i] = load[i]/2
        elif load[i] > 2100 and load[i] <= 4200:
            dg0[i] = dg1[i] = dg3[i] = load[i]/3
    elif opid[i] != 100:
        if load[i] <= 2100:
            dg0[i] = load[i]
        elif load[i] > 2100 and load[i] <= 4200:
            dg0[i] = dg1[i] = load[i]/2
    

In [97]:
# Normalize Generators
dg0n = (dg0) / (2100)
dg1n = (dg1-dg1.min()) / (2100 - dg1.min())
dg2n = (dg2-dg2.min()) / (2100 - dg2.min())
dg3n = (dg3-dg3.min()) / (2100 - dg3.min())



In [98]:
fc_dg0 = np.zeros(120960)
fc_dg1 = np.zeros(120960)
fc_dg2 = np.zeros(120960)
fc_dg3 = np.zeros(120960)

for i in range(len(dg0)):
    if dg0[i] == 0:
        fc_dg0[i] = 0
    else:
        fc_dg0[i] = (445*dg0n[i] + 47)/720
        
for i in range(len(dg1)):
    if dg1[i] == 0:
        fc_dg1[i] = 0
    else:
        fc_dg1[i] = (445*dg1n[i] + 47)/720
        
for i in range(len(dg2)):
    if dg2[i] == 0:
        fc_dg2[i] = 0
    else:
        fc_dg2[i] = (445*dg2n[i] + 47)/720
        
for i in range(len(dg3)):
    if dg3[i] == 0:
        fc_dg3[i] = 0
    else:
        fc_dg3[i] = (445*dg3n[i] + 47)/720

In [99]:
fc = sum(fc_dg0) + sum(fc_dg1) + sum(fc_dg2) + sum(fc_dg3)
print('Total fuel consumption of BASECASE uten batteri er: ', fc, 'liter')

Total fuel consumption of BASECASE uten batteri er:  46474.13143311996 liter


# SAmme med hotel load dekket av shore

In [100]:
df5 = df2.copy()

load = df5['Load'].values
dg0 = df5['engMdoPower_0_'].values
dg1 = df5['engMdoPower_1_'].values
dg2 = df5['engMdoPower_2_'].values
dg3 = df5['engMdoPower_3_'].values
opid = df5['operationId'].values
shore = [0]*120960

for i in range(len(load)):
    if opid[i] == 100:
        if load[i] <= 2100:
            dg0[i] = dg1[i] = load[i]/2
        elif load[i] > 2100 and load[i] <= 4200:
            dg0[i] = dg1[i] = dg3[i] = load[i]/3
    elif opid[i] == 6:
        shore[i] = load[i]
        dg0[i] = 0
        dg1[i] = 0
        dg3[i] = 0
    elif opid[i] != 100 and opid[i] != 6:
        if load[i] <= 2100:
            dg0[i] = load[i]
        elif load[i] > 2100 and load[i] <= 4200:
            dg0[i] = dg1[i] = load[i]/2
    

In [101]:
# Normalize Generators
dg0n = (dg0-dg0.min()) / (2100 - dg0.min())
dg1n = (dg1-dg1.min()) / (2100 - dg1.min())
dg2n = (dg2-dg2.min()) / (2100 - dg2.min())
dg3n = (dg3-dg3.min()) / (2100 - dg3.min())

In [102]:
fc_dg0 = np.zeros(120960)
fc_dg1 = np.zeros(120960)
fc_dg2 = np.zeros(120960)
fc_dg3 = np.zeros(120960)

for i in range(len(dg0)):
    if dg0[i] == 0:
        fc_dg0[i] = 0
    else:
        fc_dg0[i] = (445*dg0n[i] + 47)/720
        
for i in range(len(dg1)):
    if dg1[i] == 0:
        fc_dg1[i] = 0
    else:
        fc_dg1[i] = (445*dg1n[i] + 47)/720
        
for i in range(len(dg2)):
    if dg2[i] == 0:
        fc_dg2[i] = 0
    else:
        fc_dg2[i] = (445*dg2n[i] + 47)/720
        
for i in range(len(dg3)):
    if dg3[i] == 0:
        fc_dg3[i] = 0
    else:
        fc_dg3[i] = (445*dg3n[i] + 47)/720

In [103]:
fc = sum(fc_dg0) + sum(fc_dg1) + sum(fc_dg2) + sum(fc_dg3)
print('Total fuel consumption of BASECASE uten batteri med shore connection er: ', fc, 'liter')

Total fuel consumption of BASECASE uten batteri med shore connection er:  41795.69342657306 liter


# Emissions

In [104]:
uten_shore_connection = 2.68 * 46474 # (kg/liter) * liter
med_shore_connection = 2.68 * 41795

print('CO2-utslipp uten shore connection: ', uten_shore_connection, 'kg')
print('CO2-utslipp med shore connection: ', med_shore_connection, 'kg')

CO2-utslipp uten shore connection:  124550.32 kg
CO2-utslipp med shore connection:  112010.6 kg
